In [35]:
%load_ext autoreload
%autoreload 2

import os
from datetime import datetime
import torch
from torch.utils.data import DataLoader
from torch.optim import Adam
from torch.utils.tensorboard import SummaryWriter

from market_dynamics import bs_delta, bs_generator, bs_call_price
from data import SimulationData
from utils import call_payoff, stochastic_integral
from models import ControlNet
from train import train, test

import seaborn as sns

current_time = datetime.now().strftime('%b%d_%H-%M-%S')
comment = ""
log_dir = os.path.join('experiments', current_time + '_' + comment)
writer = SummaryWriter(log_dir)

n_simulations = 500 
initial_value = 100
sigma = 0.2
n_steps = 50
rf = 0
strike = 95
fc_dims = [16, 16]
LR = 0.01
EPOCHS = 20

bs_params = {"n_simulations": n_simulations,
             "n_steps": n_steps,
             "initial_value": initial_value,
             "sigma": sigma,
             }

price_params = {"n_steps": n_steps,
                "initial_value": initial_value,
                "sigma": sigma,
                "rf": rf,
                "strike": strike,
                }

payoff_params = {"strike": strike}

n_train = int(0.9 * n_simulations)
n_test = n_simulations - n_train

dt = SimulationData(bs_generator, bs_params, bs_call_price, price_params, call_payoff, payoff_params)
dt_train, dt_test = torch.utils.data.random_split(dt, [n_train, n_test])
data_loader = DataLoader(dt_train, batch_size=20, shuffle=True)
cn = ControlNet(n_steps, 1, fc_dims, 1)

optimizer = Adam(cn.parameters(), lr=LR)
criterion = torch.nn.MSELoss()
train(data_loader, cn, criterion , optimizer, EPOCHS, writer)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


Epoch 0: : 23batch [00:00, 42.57batch/s, loss=1.35]
Epoch 1: : 23batch [00:00, 43.50batch/s, loss=2.66]
Epoch 2: : 23batch [00:00, 43.01batch/s, loss=3.53]
Epoch 3: : 23batch [00:00, 43.60batch/s, loss=1.78]
Epoch 4: : 23batch [00:00, 43.55batch/s, loss=2.22]
Epoch 5: : 23batch [00:00, 42.80batch/s, loss=2.43] 
Epoch 6: : 23batch [00:00, 42.62batch/s, loss=1.44]
Epoch 7: : 23batch [00:00, 42.97batch/s, loss=1.45]
Epoch 8: : 23batch [00:00, 42.62batch/s, loss=2.7] 
Epoch 9: : 23batch [00:00, 43.40batch/s, loss=1.98]
Epoch 10: : 23batch [00:00, 43.58batch/s, loss=1.36]
Epoch 11: : 23batch [00:00, 43.65batch/s, loss=1.49]
Epoch 12: : 23batch [00:00, 42.73batch/s, loss=1.71]
Epoch 13: : 23batch [00:00, 43.37batch/s, loss=1.12]
Epoch 14: : 23batch [00:00, 39.46batch/s, loss=2.03]
Epoch 15: : 23batch [00:00, 44.14batch/s, loss=2.4] 
Epoch 16: : 23batch [00:00, 42.69batch/s, loss=1.51] 
Epoch 17: : 23batch [00:00, 43.58batch/s, loss=2.18]
Epoch 18: : 23batch [00:00, 41.72batch/s, loss=1.01]
E

In [36]:
def delta(x):
    return bs_delta(n_steps, x, sigma, rf, strike)

In [37]:
# Evaluation In-Sample

_ ,model_loss = test(data_loader, cn, criterion)

l = len(data_loader.dataset)
x, x_inc, payoff, price = data_loader.dataset[:l]

d_hedge = x.apply_(delta)
si = stochastic_integral(x_inc, d_hedge)
delta_loss = criterion(price + si, payoff)

print(model_loss)
print(delta_loss)

tensor(1.4785, grad_fn=<MseLossBackward>)
tensor(2.8499)


In [38]:
# Evaluation Out-Sample
data_loader = DataLoader(dt_test, batch_size=20, shuffle=True)

_ ,model_loss = test(data_loader, cn, criterion)

l = len(data_loader.dataset)
x, x_inc, payoff, price = data_loader.dataset[:l]

d_hedge = x.apply_(delta)
si = stochastic_integral(x_inc, d_hedge)
delta_loss = criterion(price + si, payoff)

print(model_loss)
print(delta_loss)

tensor(1.7275, grad_fn=<MseLossBackward>)
tensor(2.9217)
